In [1]:
import qutip as qt

In [2]:
X = qt.sigmax()
Y = qt.sigmay()

In [3]:
from q_lab_toolbox.pauli_spin_matrices import SPIN_MATRIX_DICT
def pauli_from_str(pauli_str: str):
    """Create the Pauli string operator from a Pauli string."""

    pauli_str = [SPIN_MATRIX_DICT[pauli_mat] for pauli_mat in pauli_str]

    return qt.tensor(*pauli_str)

In [4]:
pauli_from_str("XYX")

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

In [5]:
qt.tensor(X, Y, X)

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

In [6]:
import math
import multiprocessing as mp
import random as rd
import re
from itertools import product

import numpy as np
import qutip as qt
import scipy as sc
from more_itertools import distinct_permutations

def get_paulis(m, space="full"):

    pauli_single = ["I", "X", "Y", "Z"]

    if "order" in space:
        order = int(space[-1])
        num_pauli = sum(3**i * math.comb(m, i) for i in range(order + 1))
        i = 0
        pauli_list_names = []
        for k in range(order + 1):
            terms_pauli = ["XYZ" for i in range(k)] + ["I" for i in range(m - k)]
            terms_permuted = list(distinct_permutations(terms_pauli))
            for name_list in terms_permuted:
                pauli_list_names = pauli_list_names + list(product(*name_list))
    elif "random" in space:
        num_pauli = int(re.findall("\d+\Z", space)[0])
        pauli_list_names_full = list(product(pauli_single, repeat=m))
        pauli_list_names = rd.sample(pauli_list_names_full, num_pauli)
    else:
        if space != "full":
            p_print(
                'Pauli subspace "{}" not recognised, using ALL pauli matrices'.format(
                    space
                )
            )
        num_pauli = 4**m
        pauli_list_names = list(product(pauli_single, repeat=m))

    pauli_list_names = np.array(pauli_list_names)

    pauli_list = np.ndarray([num_pauli, 2**m, 2**m], dtype=np.complex_)
    id_qubit_list = np.zeros([m, num_pauli])
    for i, name in enumerate(pauli_list_names):
        pauli_list[i, :, :] = pauli_from_str(name)
        for j in range(m):
            if name[j] != "I":
                id_qubit_list[j, i] = 1

    # pauli_index_x = np.array(2**m)
    # pauli_index_y = np.array(2**m)
    # pauli_index_factor = np.array(2**m)
    n, pauli_index_x, pauli_index_y = np.where(pauli_list != 0)
    pauli_index_factor = pauli_list[n, pauli_index_x, pauli_index_y]

    pauli_index_x = pauli_index_x.reshape(num_pauli, 2**m)
    pauli_index_y = pauli_index_y.reshape(num_pauli, 2**m)
    pauli_index_factor = pauli_index_factor.reshape(num_pauli, 2**m)
    index_list = (pauli_index_x, pauli_index_y, pauli_index_factor)

    return pauli_list, pauli_list_names, id_qubit_list, index_list

In [7]:
get_paulis(m=2, space="full")

(array([[[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j]],
 
        [[ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
         [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j],
         [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j]],
 
        [[ 0.+0.j,  0.-1.j,  0.+0.j,  0.+0.j],
         [ 0.+1.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  0.-1.j],
         [ 0.+0.j,  0.+0.j,  0.+1.j,  0.+0.j]],
 
        [[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]],
 
        [[ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j],
         [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j]],
 
        [[ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j]